In [ ]:

!pip install google-search-results serpapi jina pinecone-client

/bin/bash: line 1: nvidia-smi: command not found
Python 3.11.12
/bin/bash: line 1: nvidia-smi: command not found
Python 3.11.12


In [ ]:
!pip uninstall pinecone-client -y
!pip install pinecone
!pip install pinecone --upgrade

Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0


In [ ]:
# ✅ Install latest SDK
!pip uninstall -y pinecone-client
!pip install --upgrade pinecone
!pip install pinecone

# ✅ Set keys
import os
os.environ["PINECONE_API_KEY"] = "*******************************************************"

# ✅ Setup Pinecone
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone with API key
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

index_name = "indian-tax-chatbot"
dimension = 1536

# Try creating the index only if it doesn't already exist
try:
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric="cosine",
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )
        print(f"✅ Index '{index_name}' created.")
    else:
        print(f"ℹ️ Index '{index_name}' already exists.")
except Exception as e:
    print(f"❌ Error creating/checking index: {e}")

# Connect to the index (if it exists)
index = pc.Index(index_name)

print(f"✅ Connected to index '{index_name}'.")

Found existing installation: pinecone-client 3.0.0
Uninstalling pinecone-client-3.0.0:
  Successfully uninstalled pinecone-client-3.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 5.7 MB/s eta 0:00:00
ℹ️ Index 'indian-tax-chatbot' already exists.
✅ Connected to index 'indian-tax-chatbot'.


In [ ]:
!pip install -q google-generativeai pinecone-client python-dotenv requests
!npm install -g localtunnel

⠙⠹⠸⠼⠴
changed 22 packages in 747ms
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

In [ ]:
# 1. First clean up existing installations
!pip uninstall -y pinecone-client pinecone > /dev/null 2>&1

# 2. Install the correct versions
!pip install -q pinecone-client==2.2.2  # Specific stable version

# 3. Verify installation
!pip show pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.2 MB/s eta 0:00:00
Name: pinecone-client
Version: 2.2.2
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io/
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Proprietary License
Location: /usr/local/lib/python3.11/dist-packages
Requires: dnspython, loguru, numpy, python-dateutil, pyyaml, requests, tqdm, typing-extensions, urllib3
Required-by: 


In [ ]:
# Installations (run this first)
!pip install -q google-generativeai pinecone-client python-dotenv requests
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
changed 22 packages in 4s
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼

In [ ]:
# Imports
import os
import google.generativeai as genai
from dotenv import load_dotenv
import json
from datetime import datetime
import requests
from pinecone import Pinecone, Index # Import Pinecone and Index directly

# Initialize APIs
load_dotenv()

# ===== REPLACE THESE =====
PINECONE_API_KEY = "*******************************************************"
GOOGLE_API_KEY = "*******************************************************"
SERPER_API_KEY = "*******************************************************"
# =========================

# Configure Gemini
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY) # Use PINECONE_API_KEY here
index_name = "indian-tax-chatbot" # Define index_name here
index = pc.Index(index_name)

# Check if index exists and create if necessary (You might already have this logic elsewhere)
# ... (your index creation/checking code) ...

class IndianTaxChatbot:
    def __init__(self):
        self.user_id = None
        self.current_context = {}

    def set_user(self, user_id):
        self.user_id = user_id
        self._load_user_context()

    def _load_user_context(self):
        if self.user_id:
            try:
                results = index.fetch(ids=[self.user_id])
                if results and self.user_id in results['vectors']:
                    self.current_context = results['vectors'][self.user_id]['metadata']
                else:
                    self.current_context = {
                        "income_sources": [],
                        "investments": [],
                        "deductions": [],
                        "family_status": "",
                        "dependents": 0,
                        "housing": "",
                        "tax_slab": "",
                        "previous_queries": []
                    }
            except Exception as e:
                print(f"Error loading context: {e}")
                self.current_context = {
                    "income_sources": [],
                    "investments": [],
                    "deductions": [],
                    "family_status": "",
                    "dependents": 0,
                    "housing": "",
                    "tax_slab": "",
                    "previous_queries": []
                }

    def _save_context(self):
        if self.user_id:
            try:
                embedding = genai.embed_content(
                    model="models/embedding-001",
                    content=self.user_id,
                    task_type="retrieval_document"
                )["embedding"]

                index.upsert(vectors=[{
                    "id": self.user_id,
                    "values": embedding,
                    "metadata": self.current_context
                }])
            except Exception as e:
                print(f"Error saving context: {e}")

    # ... (rest of your chatbot class code) ...


# ===== TEST THE CHATBOT =====
chatbot = IndianTaxChatbot()
chatbot.set_user("demo_user_1")

# ... (rest of your test code) ...

Exception: The official Pinecone python package has been renamed from `pinecone-client` to `pinecone`. Please remove `pinecone-client` from your project dependencies and add `pinecone` instead. See the README at https://github.com/pinecone-io/pinecone-python-client for more information on using the python SDK.

In [ ]:
# === API TEST CELL === (Add this after Cell 1)
def test_apis():
    print("=== Testing APIs ===")

    # Test Gemini
    try:
        test_response = model.generate_content("What is 2+2?")
        print(f"✅ Gemini working. Response: {test_response.text}")
    except Exception as e:
        print(f"❌ Gemini failed: {str(e)}")
        if "403" in str(e):
            print("Fix: Enable API at https://console.cloud.google.com/apis/api/generativelanguage.googleapis.com")

    # Test Pinecone
    try:
        stats = index.describe_index_stats()
        print(f"✅ Pinecone working. Stats: {stats}")
    except Exception as e:
        print(f"❌ Pinecone failed: {str(e)}")

test_apis()  # Run the tests

=== Testing APIs ===
❌ Gemini failed: name 'model' is not defined
❌ Pinecone failed: name 'index' is not defined


In [ ]:
# === CLEANUP CELL === (Run this first if errors persist)
!pip uninstall -y google-generativeai pinecone-client
!pip install -q google-generativeai pinecone-client==3.0.0
print("✅ Packages reinstalled - restart runtime and try again")

Found existing installation: google-generativeai 0.8.4
Uninstalling google-generativeai-0.8.4:
  Successfully uninstalled google-generativeai-0.8.4
Found existing installation: pinecone-client 3.0.0
Uninstalling pinecone-client-3.0.0:
  Successfully uninstalled pinecone-client-3.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 2.9 MB/s eta 0:00:00
✅ Packages reinstalled - restart runtime and try again


In [ ]:
# Installations (run this first)
!pip install -q google-generativeai pinecone-client==3.0.0 python-dotenv requests
!npm install -g localtunnel



⠙⠹⠸⠼⠴⠦⠧
changed 22 packages in 997ms
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [ ]:
# First, completely clean up existing installations
!pip uninstall -y pinecone-client pinecone > /dev/null 2>&1
!pip install -q google-generativeai pinecone python-dotenv requests

In [ ]:
# Imports
import os
import google.generativeai as genai
import json
from datetime import datetime
import requests
from pinecone import Pinecone

# ===== API KEYS =====
PINECONE_API_KEY = "*******************************************************"
GOOGLE_API_KEY = "*******************************************************"

# Initialize APIs
try:
    # Configure Gemini
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-pro')

    # Initialize Pinecone (correct way for latest version)
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index_name = "indian-tax-chatbot"

    # Check if index exists
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=768,  # Gemini embeddings size
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )
        print(f"✅ Created new index: {index_name}")
    else:
        print(f"ℹ️ Using existing index: {index_name}")

    index = pc.Index(index_name)
    print("✅ APIs initialized successfully")

except Exception as e:
    print(f"❌ Initialization failed: {str(e)}")
    raise

# Enhanced Chatbot Class
class IndianTaxChatbot:
    def __init__(self):
        self.user_id = None
        self.current_context = {
            "income_sources": [],
            "investments": [],
            "deductions": [],
            "family_status": "",
            "dependents": 0,
            "housing": "",
            "tax_slab": "",
            "previous_queries": []
        }

    def set_user(self, user_id):
        self.user_id = user_id
        self._load_user_context()

    def _load_user_context(self):
        if not self.user_id:
            return

        try:
            results = index.fetch(ids=[self.user_id])
            if results and self.user_id in results['vectors']:
                self.current_context = results['vectors'][self.user_id]['metadata']
                print(f"✅ Loaded context for user {self.user_id}")
        except Exception as e:
            print(f"⚠️ Context load error: {str(e)}")

    def _save_context(self):
        if not self.user_id:
            return

        try:
            embedding = genai.embed_content(
                model="models/embedding-001",
                content=json.dumps(self.current_context),
                task_type="retrieval_document"
            )["embedding"]

            index.upsert(vectors=[{
                "id": self.user_id,
                "values": embedding,
                "metadata": self.current_context
            }])
        except Exception as e:
            print(f"⚠️ Context save error: {str(e)}")

# === TEST CODE ===
if __name__ == "__main__":
    # Test APIs
    try:
        print("\n=== Testing APIs ===")
        # Test Gemini
        test_response = model.generate_content("What is 2+2?")
        print(f"✅ Gemini test passed. Response: {test_response.text}")

        # Test Pinecone
        stats = index.describe_index_stats()
        print(f"✅ Pinecone test passed. Index stats: {stats}")

        # Initialize and test chatbot
        print("\n=== Testing Chatbot ===")
        chatbot = IndianTaxChatbot()
        chatbot.set_user("demo_user_1")

        # Set sample profile
        chatbot.current_context.update({
            "income_sources": ["Salary", "Freelance"],
            "investments": ["PPF"],
            "family_status": "Married",
            "dependents": 1
        })

        print("✅ Chatbot initialized successfully")
        print("\nSample profile:", json.dumps(chatbot.current_context, indent=2))

    except Exception as e:
        print(f"\n❌ Testing failed: {str(e)}")
        print("Possible fixes:")
        print("1. Verify API keys are correct")
        print("2. Check API enablement at:")
        print("   - Gemini: https://console.cloud.google.com/apis/api/generativelanguage.googleapis.com")
        print("   - Pinecone: https://app.pinecone.io")
        print("3. Restart runtime and try again")

ℹ️ Using existing index: indian-tax-chatbot
✅ APIs initialized successfully

=== Testing APIs ===

❌ Testing failed: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Possible fixes:
1. Verify API keys are correct
2. Check API enablement at:
   - Gemini: https://console.cloud.google.com/apis/api/generativelanguage.googleapis.com
   - Pinecone: https://app.pinecone.io
3. Restart runtime and try again


In [ ]:
!pip uninstall -y google-generativeai > /dev/null 2>&1
!pip install -q --upgrade google-generativeai
# Imports
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec
import json

# ===== API KEYS =====
PINECONE_API_KEY = "*******************************************************"
GOOGLE_API_KEY = "*******************************************************"

# Initialize Gemini (Updated for July 2024)
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-pro-latest')  # Updated model name

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "indian-tax-chatbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"✅ Created index: {index_name}")
else:
    print(f"ℹ️ Using existing index: {index_name}")

index = pc.Index(index_name)

# Test APIs
try:
    # Test Gemini
    response = model.generate_content("What is 2+2?")
    print("✅ Gemini test passed:", response.text)

    # Test Pinecone
    stats = index.describe_index_stats()
    print("✅ Pinecone test passed:", stats)

except Exception as e:
    print(f"❌ Test failed: {str(e)}")
    if "404" in str(e):
        print("IMPORTANT: Make sure you're using the latest model name")
        print("Current correct model: 'gemini-1.5-pro-latest'")
    print("Full troubleshooting:")
    print("1. Verify API key at https://makersuite.google.com/app/apikey")
    print("2. Enable API: https://console.cloud.google.com/apis/api/generativelanguage.googleapis.com")
    print("3. Restart runtime after making changes")

ℹ️ Using existing index: indian-tax-chatbot
✅ Gemini test passed: 2 + 2 = 4

✅ Pinecone test passed: {'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


In [ ]:
# First, ensure all required imports and initializations are present
import google.generativeai as genai
from pinecone import Pinecone
import json

# Initialize Gemini (must be in the same cell or runtime)
genai.configure(api_key="*******************************************************")  # Your actual key

# Initialize Pinecone connection
pc = Pinecone(api_key="*******************************************************")
index = pc.Index("indian-tax-chatbot")

# Sample tax data to add
tax_data = [
    {
        "id": "sec80c",
        "text": "Section 80C allows ₹1.5L deduction for investments like PPF, ELSS, NSC, etc.",
        "metadata": {
            "section": "80C",
            "max_amount": 150000,
            "applicable_to": ["individual", "huf"],
            "year": 2024
        }
    },
    {
        "id": "sec80d",
        "text": "Section 80D provides health insurance deductions (₹25K-₹50K based on age)",
        "metadata": {
            "section": "80D",
            "max_amount": 50000,
            "applicable_to": ["individual", "huf"],
            "year": 2024
        }
    }
]

# Store data in Pinecone
for item in tax_data:
    try:
        # Generate embedding
        embedding = genai.embed_content(
            model="models/embedding-001",
            content=item["text"],
            task_type="retrieval_document"
        )["embedding"]

        # Upsert to Pinecone
        index.upsert(vectors=[{
            "id": item["id"],
            "values": embedding,
            "metadata": item["metadata"]
        }])
        print(f"Added: {item['id']}")
    except Exception as e:
        print(f"Failed to add {item['id']}: {str(e)}")

print(f"\n✅ Successfully added {len(tax_data)} tax provisions")
print("Current index stats:", index.describe_index_stats())

Failed to add sec80c: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 19 Apr 2025 16:04:09 GMT', 'Content-Type': 'application/json', 'Content-Length': '103', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '35', 'x-pinecone-request-id': '7387319726410024638', 'x-envoy-upstream-service-time': '17', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 768 does not match the dimension of the index 1536","details":[]}

Failed to add sec80d: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 19 Apr 2025 16:04:10 GMT', 'Content-Type': 'application/json', 'Content-Length': '103', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '20', 'x-pinecone-request-id': '4358674475686956692', 'x-envoy-upstream-service-time': '21', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 768 does not match the dimension of the index 1536","details":[]}


✅ Successfully added 2 

In [ ]:
# Clean install packages
!pip uninstall -y pinecone-client pinecone > /dev/null 2>&1
!pip install -q google-generativeai pinecone python-dotenv

# IMPORTANT: Restart runtime now (Runtime → Restart runtime)
# Click this after running this cell

In [ ]:
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec
import json

# ===== API KEYS =====
PINECONE_API_KEY = "*******************************************************"
GOOGLE_API_KEY = "*******************************************************"
# Initialize Gemini
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-pro-latest')

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "indian-tax-chatbot"

# Delete old index if exists
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# Create new index with correct dimension
pc.create_index(
    name=index_name,
    dimension=768,  # Matches Gemini's embedding size
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

index = pc.Index(index_name)
print("✅ APIs initialized and index created")
# Test Gemini embeddings
test_embed = genai.embed_content(
    model="models/embedding-001",
    content="test",
    task_type="retrieval_document"
)["embedding"]

print(f"✅ Gemini embedding dimension: {len(test_embed)}")  # Should show 768

# Test Pinecone connection
print("Pinecone index stats:", index.describe_index_stats())

✅ APIs initialized and index created
✅ Gemini embedding dimension: 768
Pinecone index stats: {'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


In [ ]:
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec
import json

# ===== API KEYS =====
PINECONE_API_KEY = "*******************************************************"
GOOGLE_API_KEY = "*******************************************************"

# Initialize Gemini
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-pro-latest')  # Updated model name

# Initialize Pinecone (WITH CORRECT DIMENSION)
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "indian-tax-chatbot"

# ==== CRITICAL CHANGE: Delete and recreate index ====
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)  # Delete old 1536-dim index
    print(f"🗑️ Deleted old index '{index_name}'")

# Create new index with 768 dimensions
pc.create_index(
    name=index_name,
    dimension=768,  # Must match Gemini's embedding-001
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
print(f"✅ Created new index '{index_name}' with 768 dimensions")

index = pc.Index(index_name)
# Test embedding dimension
test_embed = genai.embed_content(
    model="models/embedding-001",
    content="test",
    task_type="retrieval_document"
)["embedding"]

print(f"Gemini embedding dimension: {len(test_embed)}")  # Should output 768
print(f"Pinecone index dimension: {index.describe_index_stats().dimension}")  # Should match

🗑️ Deleted old index 'indian-tax-chatbot'
✅ Created new index 'indian-tax-chatbot' with 768 dimensions
Gemini embedding dimension: 768
Pinecone index dimension: 768


In [ ]:
# ====== ENTIRE SOLUTION IN ONE CELL ======
# Run this entire block together to avoid cell isolation issues

# 1. Installations
!pip install -q google-generativeai pinecone python-dotenv
print("✅ Packages installed")

# 2. Imports
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec
import json
print("✅ Libraries imported")

# 3. Initialize APIs
try:
    # Configure Gemini
    GOOGLE_API_KEY = "*******************************************************"
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-1.5-pro-latest')

    # Initialize Pinecone
    PINECONE_API_KEY = "*******************************************************"
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index_name = "indian-tax-chatbot"

    # Delete old index if exists
    if index_name in pc.list_indexes().names():
        pc.delete_index(index_name)
        print(f"🗑️ Deleted old index '{index_name}'")

    # Create new index with correct dimension
    pc.create_index(
        name=index_name,
        dimension=768,  # Must match Gemini's embedding-001
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    index = pc.Index(index_name)
    print(f"✅ Created new index '{index_name}' with 768 dimensions")

    # Verify dimensions
    test_embed = genai.embed_content(
        model="models/embedding-001",
        content="test",
        task_type="retrieval_document"
    )["embedding"]
    print(f"🔢 Gemini embedding dimension: {len(test_embed)}")
    print(f"📊 Pinecone index stats: {index.describe_index_stats()}")

    # ====== SAMPLE DATA ADDITION ======
    tax_data = [
        {
            "id": "sec80c",
            "text": "Section 80C allows ₹1.5L deduction for investments...",
            "metadata": {"section": "80C", "max_amount": 150000}
        },
        {
            "id": "sec80d",
            "text": "Section 80D provides health insurance deductions...",
            "metadata": {"section": "80D", "max_amount": 50000}
        }
    ]

    # Add to Pinecone
    for item in tax_data:
        embedding = genai.embed_content(
            model="models/embedding-001",
            content=item["text"],
            task_type="retrieval_document"
        )["embedding"]

        index.upsert(vectors=[{
            "id": item["id"],
            "values": embedding,
            "metadata": item["metadata"]
        }])

    print(f"\n📦 Added {len(tax_data)} tax provisions")
    print("Updated index stats:", index.describe_index_stats())

except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    print("Troubleshooting:")
    print("1. Restart runtime (Runtime → Restart runtime)")
    print("2. Verify API keys are valid")
    print("3. Check API enablement at:")
    print("   - https://console.cloud.google.com/apis/api/generativelanguage.googleapis.com")
  # ====== VECTOR UPLOAD FIX ======
print("\n🔄 Uploading vectors to Pinecone...")

success_count = 0
for item in tax_data:
    try:
        # Generate embedding
        embedding = genai.embed_content(
            model="models/embedding-001",
            content=item["text"],
            task_type="retrieval_document"
        )["embedding"]

        # Debug: Print embedding dimensions
        print(f"Embedding for {item['id']} length: {len(embedding)}")

        # Upsert to Pinecone with timeout
        upsert_response = index.upsert(
            vectors=[{
                "id": item["id"],
                "values": embedding,
                "metadata": item["metadata"]
            }],
            timeout=30  # Increased timeout
        )

        # Verify upsert
        if upsert_response.upserted_count == 1:
            success_count += 1
            print(f"✔ {item['id']} added successfully")
        else:
            print(f"⚠️ Failed to add {item['id']}: {upsert_response}")

    except Exception as e:
        print(f"❌ Error adding {item['id']}: {str(e)}")

# Final verification
print(f"\n🔍 Final verification...")
stats = index.describe_index_stats()
print("Namespace counts:", stats['namespaces'])
print("Total vectors:", stats['total_vector_count'])

if success_count == len(tax_data):
    print("\n🎉 All vectors added successfully!")
else:
    print(f"\n⚠️ Only {success_count}/{len(tax_data)} vectors added")
print(index.fetch(ids=["sec80c"], namespace="tax_data"))
print(f"https://app.pinecone.io/organizations/-/projects/{pc.whoami().project_name}/indexes/{index_name}/browser")

✅ Packages installed
✅ Libraries imported
🗑️ Deleted old index 'indian-tax-chatbot'
✅ Created new index 'indian-tax-chatbot' with 768 dimensions
🔢 Gemini embedding dimension: 768
📊 Pinecone index stats: {'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

📦 Added 2 tax provisions
Updated index stats: {'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

🔄 Uploading vectors to Pinecone...
Embedding for sec80c length: 768
✔ sec80c added successfully
Embedding for sec80d length: 768
✔ sec80d added successfully

🔍 Final verification...
Namespace counts: {}
Total vectors: 0

🎉 All vectors added successfully!
FetchResponse(namespace='tax_data', vectors={}, usage=None)


AttributeError: 'Pinecone' object has no attribute 'whoami'

In [ ]:
# 1. Clean install packages
!pip uninstall -y pinecone-client pinecone > /dev/null 2>&1
!pip install -q google-generativeai pinecone python-dotenv

# 2. RESTART RUNTIME NOW (Runtime → Restart runtime)
#    - This is CRUCIAL after reinstalling packages
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec

# Initialize APIs
genai.configure(api_key="*******************************************************")
pc = Pinecone(api_key="*******************************************************")

# Delete old index if exists
index_name = "indian-tax-chatbot"
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
    print(f"🗑️ Deleted old index")

# Create new index (768 dimensions)
pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
index = pc.Index(index_name)
print(f"✅ Created index with 768 dimensions")
tax_data = [
    {"id": "sec80c", "text": "Section 80C allows ₹1.5L deduction...", "metadata": {...}},
    {"id": "sec80d", "text": "Section 80D provides health insurance...", "metadata": {...}}
]

# 1. Verify embeddings first
test_embed = genai.embed_content(
    model="models/embedding-001",
    content="test",
    task_type="retrieval_document"
)["embedding"]
print(f"🔢 Embedding dimension: {len(test_embed)}")  # Must show 768

# 2. Upload with verification
for item in tax_data:
    embedding = genai.embed_content(
        model="models/embedding-001",
        content=item["text"],
        task_type="retrieval_document"
    )["embedding"]

    response = index.upsert(
        vectors=[{
            "id": item["id"],
            "values": embedding,
            "metadata": item["metadata"]
        }],
        namespace="tax_data"  # Explicit namespace
    )
    print(f"Added {item['id']} - Response: {response}")

# 3. Final verification
print("\n🔍 Final Stats:")
print(index.describe_index_stats())
print("\nSample fetch:")
print(index.fetch(ids=["sec80c"], namespace="tax_data"))

🗑️ Deleted old index
✅ Created index with 768 dimensions
🔢 Embedding dimension: 768


MetadataDictionaryExpectedError: Column `metadata` is expected to be a dictionary, found <class 'set'>

✅ Setup complete! Ready to use:
- genai model: genai.GenerativeModel(
    model_name='models/gemini-1.5-pro-latest',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)
- pinecone index: <pinecone.data.index.Index object at 0x7ecbd426a0d0>


In [ ]:
# ====== FINAL WORKING SOLUTION ======
# Run in a FRESH runtime (Runtime → Restart runtime first)

# 1. Clean installation
!pip uninstall -y pinecone-client pinecone > /dev/null 2>&1
!pip install -q google-generativeai pinecone python-dotenv

# 2. Imports
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec
from time import sleep

# 3. Initialize APIs
try:
    print("🔌 Initializing APIs...")

    # Configure Gemini
    genai.configure(api_key="*******************************************************")
    model = genai.GenerativeModel('gemini-1.5-pro-latest')

    # Initialize Pinecone
    pc = Pinecone(api_key="*******************************************************")
    index_name = "indian-tax-chatbot"

    # Delete old index if exists
    if index_name in pc.list_indexes().names():
        pc.delete_index(index_name)
        print("🗑️ Deleted old index")
        sleep(10)  # Wait for deletion to complete

    # Create new index
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print("✅ Created index (768 dim)")
    index = pc.Index(index_name)
    sleep(15)  # Critical - wait for index to be ready

    # ====== DATA UPLOAD ======
    tax_data = [
        {
            "id": "sec80c",
            "text": "Section 80C allows ₹1.5L deduction for investments in PPF, EPF, ELSS, etc.",
            "metadata": {"section": "80C", "max_amount": 150000}  # Must be dict
        },
        {
            "id": "sec80d",
            "text": "Section 80D provides health insurance deductions up to ₹50,000 for individuals.",
            "metadata": {"section": "80D", "max_amount": 50000}  # Must be dict
        }
    ]

    print("\n⬆️ Uploading vectors...")
    for item in tax_data:
        # Generate embedding
        embedding = genai.embed_content(
            model="models/embedding-001",
            content=item["text"],
            task_type="retrieval_document"
        )["embedding"]

        # Upsert with verification
        response = index.upsert(
            vectors=[{
                "id": item["id"],
                "values": embedding,
                "metadata": item["metadata"]  # Proper dictionary format
            }],
            namespace="tax_laws"  # Specific namespace
        )
        print(f"• {item['id']}: {'✅' if response.upserted_count == 1 else '❌'}")

    # ====== FINAL VERIFICATION ======
    print("\n🔎 Verifying upload...")
    sleep(10)  # Allow time for indexing

    # PROPER way to handle FetchResponse in new Pinecone client
    fetched = index.fetch(ids=["sec80c", "sec80d"], namespace="tax_laws")
    print(f"Fetched vectors: {len(fetched.vectors)}")  # Using .vectors not ['vectors']

    # Index stats
    stats = index.describe_index_stats()
    print(f"\n📊 Index stats:")
    print(f"Total vectors: {stats.total_vector_count}")  # Using dot notation
    print(f"Namespaces: {stats.namespaces}")

    # Console link (using your project name from screenshot)
    print("\n🌐 Check console:")
    print(f"https://app.pinecone.io/project/ai-agents/index/indian-tax-chatbot/browser")

except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    print("Immediate actions:")
    print("1. RESTART RUNTIME (Runtime → Restart runtime)")
    print("2. Verify metadata is dictionary format")
# Sample query code
query_text = "What is the deduction limit for health insurance?"
query_embedding = genai.embed_content(
    model="models/embedding-001",
    content=query_text,
    task_type="retrieval_query"
)["embedding"]

results = index.query(
    vector=query_embedding,
    top_k=2,
    namespace="tax_laws",
    include_metadata=True
)

for match in results.matches:
    print(f"Section {match.metadata['section']}: {match.score:.2f}")
    print(f"Details: {match.metadata}")

🔌 Initializing APIs...
🗑️ Deleted old index
✅ Created index (768 dim)

⬆️ Uploading vectors...
• sec80c: ✅
• sec80d: ✅

🔎 Verifying upload...
Fetched vectors: 2

📊 Index stats:
Total vectors: 2
Namespaces: {'tax_laws': {'vector_count': 2}}

🌐 Check console:
https://app.pinecone.io/project/ai-agents/index/indian-tax-chatbot/browser
Section 80D: 0.74
Details: {'max_amount': 50000.0, 'section': '80D'}
Section 80C: 0.64
Details: {'max_amount': 150000.0, 'section': '80C'}


In [ ]:
# ====== UPDATED INITIALIZATION CELL ======
# 1. Clean install
!pip uninstall -y pinecone-client pinecone > /dev/null 2>&1
!pip install -q google-generativeai pinecone python-dotenv requests

# 2. Imports
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec
from datetime import datetime
import time

# 3. Initialize APIs
genai.configure(api_key="*******************************************************")  # Your key
model = genai.GenerativeModel('gemini-1.5-pro-latest')

pc = Pinecone(api_key="*******************************************************")  # Your key
index_name = "indian-tax-chatbot"

# 4. Setup Pinecone Index
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
    time.sleep(10)

pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
index = pc.Index(index_name)
time.sleep(15)

print("✅ Setup complete! Ready to use:")
print("- genai model:", model)
print("- pinecone index:", index)
# Step 1: Create user profile structure
def create_user_profile(user_id):
    return {
        "user_id": user_id,  # Unique identifier (hash of email/phone)
        "income_sources": [],
        "investments": {},
        "family_status": "",
        "housing_loan": False,
        "previous_discussions": []  # Will store vector IDs
    }

# Step 2: Profile update function
def update_profile(profile, new_data):
    for key, value in new_data.items():
        if key in profile:
            profile[key] = value
    return profile
# Step 3: Modified upsert with user context
def store_conversation(user_id, conversation_text, metadata={}):
    embedding = genai.embed_content(
        model="models/embedding-001",
        content=conversation_text,
        task_type="retrieval_document"
    )["embedding"]

    vector_id = f"conv_{int(time.time())}"

    index.upsert(vectors=[{
        "id": vector_id,
        "values": embedding,
        "metadata": {
            "user_id": user_id,
            "timestamp": datetime.now().isoformat(),
            **metadata
        }
    }], namespace="user_conversations")

    return vector_id
# Step 4: Recall previous discussions
def get_previous_context(user_id, current_query, top_k=3):
    # Get embedding of current query
    query_embed = genai.embed_content(
        model="models/embedding-001",
        content=current_query,
        task_type="retrieval_query"
    )["embedding"]

    # Query with user filter
    results = index.query(
        vector=query_embed,
        top_k=top_k,
        namespace="user_conversations",
        filter={"user_id": {"$eq": user_id}},
        include_metadata=True
    )

    return [match.metadata for match in results.matches]
# Step 5: Deduction analysis tool
def analyze_deductions(user_profile):
    applicable = []

    # Check 80C eligibility
    if any(inv in user_profile["investments"] for inv in ["PPF", "ELSS", "FD"]):
        applicable.append({
            "section": "80C",
            "max_amount": 150000,
            "description": "Investments in specified instruments"
        })

    # Check 80D eligibility
    if user_profile.get("health_insurance"):
        applicable.append({
            "section": "80D",
            "max_amount": 50000 if user_profile["age"] > 60 else 25000,
            "description": "Health insurance premium"
        })

    return applicable
# Step 6: Real-time tax updates
def get_tax_updates():
    url = "https://newsapi.org/v2/everything?q=india+tax+law&sortBy=publishedAt&apiKey=YOUR_KEY"
    response = requests.get(url)
    articles = response.json().get("articles", [])

    # Process and store relevant updates
    for article in articles[:5]:  # Limit to 5 most recent
        store_conversation(
            user_id="system",
            conversation_text=article["title"] + "\n" + article["description"],
            metadata={
                "source": "NewsAPI",
                "published_at": article["publishedAt"],
                "type": "tax_update"
            }
        )
# Step 7: Main conversation processor
def handle_query(user_id, query):
    # Step 1: Get user profile (from DB in production)
    profile = get_user_profile(user_id)

    # Step 2: Recall relevant context
    context = get_previous_context(user_id, query)

    # Step 3: Get latest updates
    tax_updates = get_tax_updates()

    # Step 4: Generate response
    prompt = f"""
    User Profile: {profile}
    Previous Discussions: {context}
    Latest Updates: {tax_updates}

    New Query: {query}
    """

    response = model.generate_content(prompt)

    # Step 5: Store this interaction
    store_conversation(user_id, f"Q: {query}\nA: {response.text}")

    return response.text
# Silent setup - no print needed
user_profiles = {}

def get_user_profile(user_id):
    if user_id not in user_profiles:
        user_profiles[user_id] = {
            "user_id": user_id,
            "income_sources": [],
            "investments": {},
            "family_status": "",
            "previous_discussions": []
        }
    return user_profiles[user_id]
def silent_upsert(user_id, text, metadata={}):
    embedding = genai.embed_content(
        model="models/embedding-001",
        content=text,
        task_type="retrieval_document"
    )["embedding"]

    vector_id = f"mem_{user_id}_{int(time.time())}"
    index.upsert(
        vectors=[{
            "id": vector_id,
            "values": embedding,
            "metadata": {"user_id": user_id, **metadata}
        }],
        namespace="tax_memory"
    )
    return vector_id  # No print
def get_context(user_id, query, top_k=2):
    query_embed = genai.embed_content(
        model="models/embedding-001",
        content=query,
        task_type="retrieval_query"
    )["embedding"]

    return index.query(
        vector=query_embed,
        top_k=top_k,
        filter={"user_id": {"$eq": user_id}},
        namespace="tax_memory",
        include_metadata=True
    )
# Tool 1: Deduction Calculator
def calculate_deductions(profile):
    deductions = []
    if profile["investments"].get("80C"):
        deductions.append(("80C", min(150000, profile["investments"]["80C"])))
    return deductions

# Tool 2: Update Checker
def check_for_updates():
    # Placeholder for API call
    return [{"section": "80C", "change": "New PPF limits"}]
# ====== LLM INTEGRATION (NEW CELL) ======
# 1. First-time setup (run only once)
!pip install -q google-generativeai
import google.generativeai as genai
from datetime import datetime

# 2. Initialize Gemini Model
genai.configure(api_key="*******************************************************")
tax_agent = genai.GenerativeModel('gemini-1.5-pro-latest')

# 3. Fixed Handler Function
def tax_chatbot(user_id, query):
    """Handles complete query processing"""
    try:
        # Get user context
        profile = get_user_profile(user_id)
        context = get_previous_context(user_id, query)
        deductions = analyze_deductions(profile)

        # Generate response
        response = tax_agent.generate_content(
            f"""**Act as Indian Tax Expert**\n
            User Profile:\n{profile}\n
            Past Discussions:\n{context}\n
            Eligible Deductions:\n{deductions}\n\n
            Query: "{query}"\n
            Give detailed, personalized answer with section references.
            """
        )

        # Store conversation
        store_conversation(
            user_id=user_id,
            conversation_text=f"Q: {query}\nA: {response.text}",
            metadata={"type": "tax_advice"}
        )

        return response.text

    except Exception as e:
        print(f"⚠️ Error: {str(e)}")
        return "Unable to process request. Please try again."

# 4. Test Function
def test_flow():
    # Setup test user
    user_id = "test_user_001"
    profile = {
        "investments": {"PPF": 150000, "ELSS": 50000},
        "health_insurance": True,
        "age": 45
    }
    update_profile(get_user_profile(user_id), profile)

    # Test query
    query = "How much tax can I save under 80C and 80D?"
    return tax_chatbot(user_id, query)

# 5. Execute Test
test_response = test_flow()
print("💬 Tax Bot Response:")
print(test_response)
# Add to your profile structure
def add_health_insurance(user_id, details):
    profile = get_user_profile(user_id)
    profile["health_insurance"] = {
        "self_covered": details.get("self", False),
        "parents_covered": details.get("parents", False),
        "premium_amount": details.get("amount", 0)
    }
    return profile
def get_budget_updates():
    """Fetch latest budget announcements"""
    import requests
    url = "https://newsapi.org/v2/everything?q=india+union+budget+2025&apiKey=YOUR_KEY"
    response = requests.get(url)
    return [article["title"] for article in response.json().get("articles",[])[:3]]
def get_chat_history(user_id):
    return index.query(
        vector=[0]*768,  # Dummy vector
        filter={"user_id": {"$eq": user_id}},
        namespace="tax_conversations",
        include_metadata=True,
        top_k=5
    )
def get_latest_tax_updates():
    """Fetch real-time tax law changes using Serper API"""
    import requests
    import json

    url = "https://google.serper.dev/search"
    payload = json.dumps({
        "q": "site:incometaxindia.gov.in latest notification after:2025-03-01",
        "gl": "in"
    })
    headers = {
        'X-API-KEY': '*******************************************************',  # Get free tier key at serper.dev
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    updates = [item["title"] for item in response.json().get("organic", [])[:3]]
    return {"source": "IncomeTaxIndia.gov.in", "updates": updates}
def handle_query(user_id, query):
    try:
        profile = get_user_profile(user_id)
        context = get_previous_context(user_id, query)
        updates = get_tax_updates()  # Using our new tool

        prompt = f"""**Act as Tax Expert with Latest Info**

        Latest Updates: {updates}

        User Profile: {profile}

        Conversation History: {context}

        Question: {query}

        Give specific advice referencing:
        1. Relevant sections from user profile
        2. Any applicable recent changes
        3. Precise calculation if needed"""

        response = model.generate_content(prompt)

        # Store interaction
        store_conversation(
            user_id=user_id,
            conversation_text=f"Q: {query}\nA: {response.text}",
            metadata={
                "type": "tax_advice",
                "sources": [updates.get("source")]
            }
        )

        return response.text

    except Exception as e:
        error_msg = f"Tax query failed: {str(e)}"
        print(error_msg)
        return error_msg
# Test the new functionality
print("📢 Latest Updates:", get_latest_tax_updates())
test_response = tax_chatbot("test_user_001",
    "How do the recent notifications affect my 80D claims?")
print(test_response)
try:
    updates = get_latest_tax_updates()
except Exception as e:
    updates = {"error": str(e)}
from datetime import datetime, timedelta

update_cache = {"last_fetched": None, "data": None}

def get_cached_updates():
    if update_cache["last_fetched"] and \
       (datetime.now() - update_cache["last_fetched"]) < timedelta(hours=1):
        return update_cache["data"]
    update_cache["data"] = get_latest_tax_updates()
    update_cache["last_fetched"] = datetime.now()
    return update_cache["data"]
# ====== REAL-TIME TAX UPDATES TOOL ======
update_cache = {
    "last_fetched": None,
    "data": None,
    "cache_duration": timedelta(hours=1)  # Refresh every hour
}

def get_tax_updates():
    """Get latest tax notifications with caching and error handling"""
    try:
        # Return cached data if still valid
        if update_cache["last_fetched"] and \
           (datetime.now() - update_cache["last_fetched"]) < update_cache["cache_duration"]:
            return update_cache["data"]

        # Fetch fresh data
        url = "https://google.serper.dev/search"
        payload = json.dumps({
            "q": "site:incometaxindia.gov.in notification after:{date}".format(
                date=(datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")),
            "gl": "in",
            "num": 5
        })
        headers = {
            'X-API-KEY': '*******************************************************',
            'Content-Type': 'application/json'
        }

        response = requests.post(url, headers=headers, data=payload, timeout=10)
        response.raise_for_status()

        # Process and cache results
        updates = [{
            "title": item["title"],
            "link": item["link"],
            "snippet": item.get("snippet", "")
        } for item in response.json().get("organic", [])[:3]]

        update_cache["data"] = {
            "source": "IncomeTaxIndia.gov.in",
            "last_checked": datetime.now().isoformat(),
            "updates": updates
        }
        update_cache["last_fetched"] = datetime.now()

        return update_cache["data"]

    except Exception as e:
        print(f"⚠️ Update fetch failed: {str(e)}")
        return {
            "error": "Could not fetch updates",
            "cached_data": update_cache["data"]
        }
# Safe test without exposing API calls
test_query = "How do recent notifications affect NRI taxation?"
test_response = handle_query("test_user_007", test_query)

print("🔍 Test Results:")
print("First 200 chars:", test_response[:200] + "...")
print("\nUpdate Cache Status:", bool(update_cache["data"]))

✅ Setup complete! Ready to use:
- genai model: genai.GenerativeModel(
    model_name='models/gemini-1.5-pro-latest',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)
- pinecone index: <pinecone.data.index.Index object at 0x7fdb7ddf60d0>
💬 Tax Bot Response:
Hello,

Based on the information you've provided, let's explore your potential tax savings under sections 80C and 80D of the Income Tax Act, 1961.

**Section 80C: Deductions for Investments**

Section 80C allows for deductions up to ₹150,000 per year for investments in specified instruments. You've mentioned investments in PPF (Public Provident Fund) and ELSS (Equity Linked Savings Scheme).

* **PPF:** Your contribution of ₹150,000 to PPF is eligible for deduction under 80C.
* **ELSS:** Your investment of ₹50,000 in ELSS is also eligible for deduction under 80C.

Since the combined investment in PPF and ELSS is ₹200,000, and the maximum deduction limit under 80C 

In [ ]:
# ====== COMPLETE TAX CHATBOT SOLUTION ======
# Run in a FRESH runtime (Runtime → Restart runtime first)

# 1. Installations and Imports
!pip uninstall -y pinecone-client pinecone > /dev/null 2>&1
!pip install -q google-generativeai pinecone python-dotenv requests fastapi uvicorn python-multipart prometheus_client pdfminer.six

import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec
from datetime import datetime, timedelta
import time
import requests
import json
from cryptography.fernet import Fernet
import base64
import os
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from collections import defaultdict
from functools import lru_cache
from fastapi import FastAPI, Depends, HTTPException
import uvicorn
from fastapi.security import OAuth2PasswordBearer
from prometheus_client import start_http_server, Counter
import sqlite3
from dotenv import load_dotenv
from pdfminer.high_level import extract_text
import asyncio

# 2. Initialize APIs
try:
    print("🔌 Initializing APIs...")
    load_dotenv()

    # Configure Gemini
    genai.configure(api_key=os.getenv("GEMINI_API_KEY", "*******************************************************"))
    model = genai.GenerativeModel('gemini-1.5-pro-latest')

    # Initialize Pinecone
    pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY", "*******************************************************"))
    index_name = "indian-tax-chatbot"

    # Setup Pinecone Index
    if index_name in pc.list_indexes().names():
        pc.delete_index(index_name)
        print("🗑️ Deleted old index")
        time.sleep(10)

    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    index = pc.Index(index_name)
    time.sleep(15)
    print("✅ Created index (768 dim)")

    # ====== CORE FUNCTIONS ======
    user_profiles = {}  # In-memory DB
    active_sessions = {}  # For session management
    METRICS = {
        'queries_processed': defaultdict(int),
        'response_time': defaultdict(float),
        'errors': defaultdict(int)
    }

    def get_user_profile(user_id):
        if user_id not in user_profiles:
            user_profiles[user_id] = {
                "user_id": user_id,
                "income_sources": [],
                "investments": {},
                "family_status": "",
                "housing_loan": False,
                "age": None,
                "previous_discussions": []
            }
        return user_profiles[user_id]

    def update_profile(profile, new_data):
        for key, value in new_data.items():
            if key in profile:
                profile[key] = value
        return profile

    def store_conversation(user_id, text, metadata={}):
        embedding = genai.embed_content(
            model="models/embedding-001",
            content=text,
            task_type="retrieval_document"
        )["embedding"]

        index.upsert(
            vectors=[{
                "id": f"conv_{datetime.now().timestamp()}",
                "values": embedding,
                "metadata": {
                    "user_id": user_id,
                    "timestamp": datetime.now().isoformat(),
                    **{k: str(v) for k,v in metadata.items()}
                }
            }],
            namespace="tax_conversations"
        )

    def get_previous_context(user_id, query, top_k=3):
        query_embed = genai.embed_content(
            model="models/embedding-001",
            content=query,
            task_type="retrieval_query"
        )["embedding"]

        results = index.query(
            vector=query_embed,
            top_k=top_k,
            namespace="tax_conversations",
            filter={"user_id": {"$eq": user_id}},
            include_metadata=True
        )
        return [match.metadata for match in results.matches]

    # ====== TAX TOOLS ======
    def analyze_deductions(profile):
        applicable = []
        if any(inv in profile["investments"] for inv in ["PPF", "ELSS", "FD"]):
            applicable.append({
                "section": "80C",
                "max_amount": 150000,
                "description": "Investments in specified instruments"
            })
        if profile.get("health_insurance"):
            applicable.append({
                "section": "80D",
                "max_amount": 50000 if profile["age"] > 60 else 25000,
                "description": "Health insurance premium"
            })
        return applicable

    update_cache = {
        "last_fetched": None,
        "data": None,
        "cache_duration": timedelta(hours=1)
    }

    def get_tax_updates():
        try:
            if update_cache["last_fetched"] and \
               (datetime.now() - update_cache["last_fetched"]) < update_cache["cache_duration"]:
                return update_cache["data"]

            url = "https://google.serper.dev/search"
            payload = json.dumps({
                "q": "site:incometaxindia.gov.in notification after:{date}".format(
                    date=(datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")),
                "gl": "in"
            })
            headers = {
                'X-API-KEY': os.getenv("SERPER_API_KEY", "*******************************************************"),
                'Content-Type': 'application/json'
            }

            response = requests.post(url, headers=headers, data=payload, timeout=10)
            response.raise_for_status()

            updates = [{
                "title": item["title"],
                "link": item["link"],
                "snippet": item.get("snippet", "")
            } for item in response.json().get("organic", [])[:3]]

            update_cache["data"] = {
                "source": "IncomeTaxIndia.gov.in",
                "updates": updates,
                "last_checked": datetime.now().isoformat()
            }
            update_cache["last_fetched"] = datetime.now()

            return update_cache["data"]

        except Exception as e:
            print(f"⚠️ Update fetch failed: {str(e)}")
            return {"error": "Could not fetch updates"}

    # ====== ADVANCED TAX TOOLS ======
    def calculate_tax_savings(profile, deductions):
        tax_slabs = {
            "5%": 300000,
            "10%": 500000,
            "15%": 750000,
            "20%": 1000000,
            "30%": float('inf')
        }

        taxable_income = profile.get("annual_income", 0)
        current_slab = next((k for k,v in tax_slabs.items() if taxable_income <= v), "30%")
        slab_rate = int(current_slab.strip('%')) / 100

        return {
            ded["section"]: (
                ded["max_amount"],
                ded["max_amount"] * slab_rate
            ) for ded in deductions
        }

    def optimize_investments(profile, available_cash):
        suggestions = []

        # 80C Optimization
        remaining_80c = 150000 - profile["investments"].get("80C", 0)
        if remaining_80c > 0 and available_cash > 0:
            invest = min(remaining_80c, available_cash)
            suggestions.append(
                f"Invest ₹{invest} in PPF/ELSS to maximize 80C (remaining limit: ₹{remaining_80c})"
            )
            available_cash -= invest

        # 80D Optimization
        if profile.get("health_insurance") and available_cash > 0:
            limit = 50000 if profile["age"] > 60 else 25000
            current = profile["health_insurance"].get("premium_amount", 0)
            remaining_80d = limit - current

            if remaining_80d > 0:
                invest = min(remaining_80d, available_cash)
                suggestions.append(
                    f"Top-up health insurance by ₹{invest} (80D remaining limit: ₹{remaining_80d})"
                )

        return suggestions or ["No optimization needed - all limits utilized"]

    # ====== ENCRYPTION TOOLS ======
    class SecureEncryptor:
        def __init__(self, secret_key=None):
            self.key = self._derive_key(secret_key or os.getenv("ENCRYPTION_SECRET", "your-strong-secret-key"))

        def _derive_key(self, password: str):
            salt = b'salt_'  # Change this in production!
            kdf = PBKDF2HMAC(
                algorithm=hashes.SHA256(),
                length=32,
                salt=salt,
                iterations=100000
            )
            return base64.urlsafe_b64encode(kdf.derive(password.encode()))

        def encrypt(self, plaintext: str):
            f = Fernet(self.key)
            return f.encrypt(plaintext.encode()).decode()

        def decrypt(self, ciphertext: str):
            f = Fernet(self.key)
            return f.decrypt(ciphertext.encode()).decode()

        def rotate_keys(self):
            new_key = Fernet.generate_key()
            return new_key

    encryptor = SecureEncryptor()

    # ====== SESSION MANAGEMENT ======
    def start_session(user_id):
        active_sessions[user_id] = {
            "start_time": datetime.now(),
            "message_count": 0,
            "active_tools": []
        }
        return f"Session started for {user_id}"

    def end_session(user_id):
        if user_id in active_sessions:
            session_data = active_sessions.pop(user_id)
            store_conversation(
                user_id=user_id,
                text=f"Session ended after {session_data['message_count']} messages",
                metadata={"type": "session_log"}
            )

    def get_all_active_sessions():
        return {
            "total_sessions": len(active_sessions),
            "active_users": list(active_sessions.keys()),
            "oldest_session": min(sess["start_time"] for sess in active_sessions.values())
        }

    # ====== SECURE PROFILE MANAGEMENT ======
    def add_financial_details(user_id, details):
        profile = get_user_profile(user_id)
        sensitive_fields = ["pan_number", "bank_account", "aadhaar"]

        for field in sensitive_fields:
            if field in details:
                details[field] = encryptor.encrypt(details[field])

        profile.update(details)
        return profile

    def get_decrypted_details(user_id, fields):
        profile = get_user_profile(user_id)
        return {
            field: encryptor.decrypt(profile[field])
            for field in fields
            if field in profile
        }

    # ====== MENU SYSTEM ======
    def show_main_menu():
        return """🔍 How can I help with your taxes today?
        1. Get deduction advice
        2. Calculate tax liability
        3. Optimize investments
        4. Check latest tax updates
        5. Manage profile"""

    def handle_menu_selection(choice: int, user_id: str):
        menu_actions = {
            1: lambda: tax_chatbot(user_id, "Get deduction advice"),
            2: lambda: tax_chatbot(user_id, "Calculate my tax liability"),
            3: lambda: tax_chatbot(user_id, "Optimize my investments"),
            4: lambda: get_tax_updates(),
            5: lambda: "Profile management selected"
        }
        return menu_actions.get(choice, lambda: "Invalid choice")()

    # ====== PERFORMANCE TOOLS ======
    @lru_cache(maxsize=1000)
    def get_tax_slab(income):
        tax_slabs = {
            (0, 300000): "5%",
            (300001, 500000): "10%",
            (500001, 750000): "15%",
            (750001, 1000000): "20%",
            (1000001, float('inf')): "30%"
        }
        for (lower, upper), slab in tax_slabs.items():
            if lower <= income <= upper:
                return slab
        return "30%"

    # ====== METRICS SETUP ======
    REQUEST_COUNT = Counter('chatbot_requests_total', 'Total number of chatbot requests')
    ERROR_COUNT = Counter('chatbot_errors_total', 'Total number of chatbot errors')
    RESPONSE_TIME = Counter('chatbot_response_time_seconds', 'Total response time in seconds')

    # ====== MAIN CHATBOT HANDLER ======
    def tax_chatbot(user_id, query):
        try:
            METRICS['queries_processed'][user_id] += 1
            start_time = time.time()

            if user_id not in active_sessions:
                start_session(user_id)

            active_sessions[user_id]["message_count"] += 1

            profile = get_user_profile(user_id)
            context = get_previous_context(user_id, query)
            updates = get_tax_updates()
            deductions = analyze_deductions(profile)

            prompt = f"""**Tax Expert Analysis**\n
            User Profile: {profile}\n
            Latest Updates: {updates}\n
            Previous Discussions: {context}\n\n
            Question: {query}\n\n
            Provide:
            1. Relevant sections
            2. Tax savings calculation
            3. Optimization suggestions"""

            needs_calculations = any(word in query.lower() for word in ["save", "calculate", "optimize"])

            if needs_calculations:
                calculations = calculate_tax_savings(profile, deductions)
                optimizations = optimize_investments(profile, 100000)

                prompt += "\n\nAdditional Instructions:\n"
                prompt += "- Include exact tax savings calculations\n"
                prompt += "- Provide specific investment optimization suggestions\n"
                prompt += "- Format numbers clearly with ₹ symbol"

            response = model.generate_content(prompt)
            response_text = response.text

            if needs_calculations:
                calculations = calculate_tax_savings(profile, deductions)
                optimizations = optimize_investments(profile, 100000)

                response_text += "\n\n🔢 **Detailed Calculations:**\n"
                for section, (amount, savings) in calculations.items():
                    response_text += f"- {section}: ₹{amount} → Saves ₹{savings:.0f}/year\n"

                response_text += "\n💡 **Optimization Suggestions:**\n"
                response_text += "\n".join(f"- {s}" for s in optimizations)

            store_conversation(
                user_id=user_id,
                text=f"Q: {query}\nA: {response_text}",
                metadata={
                    "type": "tax_advice",
                    "session_id": hash(user_id),
                    "message_num": active_sessions[user_id]["message_count"]
                }
            )

            METRICS['response_time'][user_id] = time.time() - start_time
            return response_text

        except Exception as e:
            METRICS['errors'][str(e)] += 1
            error_msg = f"Error processing query: {str(e)}"
            print(error_msg)
            return error_msg

    # ====== WEB SERVER SETUP ======
    app = FastAPI()
    start_http_server(8001)

    oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

    async def get_current_user(token: str = Depends(oauth2_scheme)):
        return {"user_id": "test_user"}

    @app.post("/chat")
    async def chat_endpoint(user_id: str, query: str):
        start_time = time.time()
        REQUEST_COUNT.inc()
        try:
            response = tax_chatbot(user_id, query)
            RESPONSE_TIME.inc(time.time() - start_time)
            return {"response": response}
        except Exception as e:
            ERROR_COUNT.inc()
            raise HTTPException(status_code=500, detail=str(e))

    @app.post("/async-chat")
    async def async_chat(user_id: str, query: str):
        return await asyncio.to_thread(tax_chatbot, user_id, query)

    # ====== DATABASE SETUP ======
    def init_db():
        conn = sqlite3.connect('tax_chatbot.db')
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS users
                    (user_id TEXT PRIMARY KEY, profile TEXT)''')
        conn.commit()
        conn.close()

    # ====== FORM 16 PROCESSOR ======
    def extract_form16_data(pdf_path):
        text = extract_text(pdf_path)
        return model.generate_content(f"""
        Extract tax details from this Form 16:
        {text}

        Return JSON with: employer_name, salary_breakdown, tds_details
        """)

    # ====== TRANSLATION SUPPORT ======
    def translate_response(response, target_language="hi"):
        if target_language == "en":
            return response
        return model.generate_content(f"""
        Translate this tax advice to {target_language}:
        {response}

        Maintain technical accuracy while translating.
        """)

    # ====== TEST FLOW ======
    print("\n🚀 Testing complete flow...")
    init_db()

    # Setup test user
    test_user = "test_user_001"
    update_profile(
        get_user_profile(test_user),
        {
            "investments": {"PPF": 150000, "ELSS": 50000},
            "health_insurance": True,
            "age": 45,
            "annual_income": 900000
        }
    )

    # Test menu system
    print("\n📋 Testing menu system:")
    print(show_main_menu())
    print(handle_menu_selection(1, test_user)[:200] + "...")

    # Test basic query
    print("\n💬 Testing basic tax advice:")
    basic_response = tax_chatbot(test_user, "How do recent notifications affect my 80D claims with ₹25k premium?")
    print(basic_response[:500] + "...")

    # Test optimization query
    print("\n🧪 Testing optimization features:")
    optimization_response = tax_chatbot(test_user, "How can I optimize ₹1,00,000 for maximum tax savings?")
    print(optimization_response)

    # Test encryption
    print("\n🔒 Testing encryption:")
    user_data = {
        "pan_number": "ABCDE1234F",
        "bank_account": "1234567890"
    }
    updated = add_financial_details("secure_user", user_data)
    decrypted = get_decrypted_details("secure_user", ["pan_number"])
    print(f"Original: {user_data['pan_number']}")
    print(f"Encrypted: {updated['pan_number']}")
    print(f"Decrypted: {decrypted['pan_number']}")

    print("\n✅ All tests completed successfully!")

    # Start server (commented for notebook)
    # if __name__ == "__main__":
    #     uvicorn.run(app, host="0.0.0.0", port=8000)

except Exception as e:
    print(f"\n❌ Critical error: {str(e)}")
    print("Immediate actions:")
    print("1. RESTART RUNTIME (Runtime → Restart runtime)")
    print("2. Check API keys are valid")

🔌 Initializing APIs...
🗑️ Deleted old index
✅ Created index (768 dim)

🚀 Testing complete flow...

📋 Testing menu system:
🔍 How can I help with your taxes today?
        1. Get deduction advice
        2. Calculate tax liability
        3. Optimize investments
        4. Check latest tax updates
        5. Manage profile
## Tax Deduction Advice based on Provided Information

**1. Relevant Sections:**

Based on the user profile, the following sections are relevant for tax deductions:

* **Section 80C:** Deductions for ...

💬 Testing basic tax advice:
Here's an analysis based on the provided information:

**1. Relevant Sections:**

The user's question pertains to Section 80D of the Income Tax Act, which deals with deductions for health insurance premiums.  Specifically, the question asks about the impact of *recent notifications*.  While the provided updates mention new notifications (e.g., Notification No. 30/2025), the snippets don't offer details about changes specifically to Section 

In [ ]:
%%writefile main.py
from fastapi import FastAPI
app = FastAPI()

# PASTE YOUR ENTIRE EXISTING FASTAPI CODE HERE
# (from the working Colab cells where you defined your tax chatbot)


Writing main.py


In [ ]:
%%writefile requirements.txt
fastapi==0.109.1
uvicorn==0.27.0
python-multipart==0.0.6
google-generativeai==0.3.2
pinecone-client==3.0.2
python-dotenv==1.0.0
cryptography==42.0.2


Writing requirements.txt


In [ ]:
!ls  # List all files in current Colab directory

main.py  requirements.txt  sample_data


In [ ]:
!ls -l  # Check exact capitalization (Main.py vs main.py)

total 12
-rw-r--r-- 1 root root  160 Apr 21 19:10 main.py
-rw-r--r-- 1 root root  149 Apr 21 19:10 requirements.txt
drwxr-xr-x 1 root root 4096 Apr 17 13:36 sample_data


In [ ]:
# Create zip by explicitly listing files
!zip tax_chatbot_project.zip main.py requirements.txt

# Alternative if above fails:
!zip tax_chatbot_project.zip *.py *.txt

  adding: main.py (deflated 15%)
  adding: requirements.txt (deflated 23%)
updating: main.py (deflated 15%)
updating: requirements.txt (deflated 23%)


In [ ]:
!ls  # Should show tax_chatbot_project.zip

main.py  requirements.txt  sample_data	tax_chatbot_project.zip


In [ ]:
!zip -r tax_chatbot_project.zip main.py requirements.txt
from google.colab import files
files.download('tax_chatbot_project.zip')

updating: main.py (deflated 15%)
updating: requirements.txt (deflated 23%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install fastapi uvicorn python-multipart pydantic
!pip list | grep -E "fastapi|uvicorn|pydantic"
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional
from datetime import datetime
import logging

# Initialize app
app = FastAPI(
    title="Indian Tax Chatbot API",
    description="API for personalized tax guidance with memory",
    version="1.0.0"
)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# --- Data Models ---
class UserProfile(BaseModel):
    user_id: str
    age: int
    annual_income: float
    investments: dict = {}
    health_insurance: bool = False
    housing_loan: bool = False

class TaxQuery(BaseModel):
    question: str
    context: Optional[str] = None

class OptimizationRequest(BaseModel):
    available_amount: float
    financial_goals: List[str] = []

# --- Database Mock ---
user_profiles_db = {}
conversation_history = {}

# --- Core Endpoints ---
@app.post("/register", summary="Register new user")
async def register_user(profile: UserProfile):
    """Store user profile with financial details"""
    user_profiles_db[profile.user_id] = profile.dict()
    logger.info(f"New user registered: {profile.user_id}")
    return {"status": "success", "user_id": profile.user_id}

@app.get("/profile/{user_id}", response_model=UserProfile)
async def get_profile(user_id: str):
    """Retrieve user profile"""
    if user_id not in user_profiles_db:
        raise HTTPException(status_code=404, detail="User not found")
    return user_profiles_db[user_id]

@app.post("/ask", summary="Get tax advice")
async def ask_tax_question(user_id: str, query: TaxQuery):
    """
    Main tax advice endpoint with conversation memory
    """
    profile = user_profiles_db.get(user_id, {})

    # ACTUAL TAX LOGIC IMPLEMENTATION
    tax_slabs = [
        (0, 300000, 0),
        (300001, 600000, 0.05),
        (600001, 900000, 0.10),
        (900001, 1200000, 0.15),
        (1200001, 1500000, 0.20),
        (1500001, float('inf'), 0.30)
    ]

    # Calculate deductions
    deductions = {
        '80C': min(150000, profile.get('investments', {}).get('80C', 0)),
        '80D': 50000 if profile.get('age', 0) > 60 else 25000 if profile.get('health_insurance') else 0,
        'NPS': min(50000, profile.get('investments', {}).get('NPS', 0))
    }

    response = {
        "answer": f"For {user_id}: {query.question}",
        "suggestions": [
            f"80C: ₹{deductions['80C']} available",
            f"80D: ₹{deductions['80D']} deductible",
            f"NPS: ₹{deductions['NPS']} contribution"
        ],
        "timestamp": datetime.now().isoformat()
    }

    # Store conversation
    if user_id not in conversation_history:
        conversation_history[user_id] = []
    conversation_history[user_id].append({
        "question": query.question,
        "response": response,
        "timestamp": datetime.now().isoformat()
    })

    return response

@app.post("/optimize", summary="Tax optimization planner")
async def optimize_investments(user_id: str, request: OptimizationRequest):
    """ACTUAL OPTIMIZATION LOGIC"""
    profile = user_profiles_db.get(user_id, {})
    suggestions = []
    remaining = request.available_amount

    # 80C Optimization
    used_80c = profile.get('investments', {}).get('80C', 0)
    remaining_80c = max(0, 150000 - used_80c)
    invest_80c = min(remaining_80c, remaining)
    if invest_80c > 0:
        suggestions.append(f"Invest ₹{invest_80c} in ELSS (80C)")
        remaining -= invest_80c

    # NPS Optimization
    if remaining > 0:
        invest_nps = min(50000, remaining)
        suggestions.append(f"Invest ₹{invest_nps} in NPS (80CCD(1B))")
        remaining -= invest_nps

    return {
        "optimization_plan": suggestions,
        "estimated_savings": (request.available_amount - remaining) * 0.3,
        "remaining_amount": remaining
    }

# --- Technical Endpoints ---
@app.get("/health")
async def health_check():
    return {"status": "healthy"}

# --- CORS Setup ---
from fastapi.middleware.cors import CORSMiddleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)


fastapi                               0.115.12
pydantic                              2.11.3
pydantic_core                         2.33.1
uvicorn                               0.34.2


In [ ]:
!uvicorn main:app --reload --host 0.0.0.0 --port 8000

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [25519] using StatReload
INFO:     Started server process [25525]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25525]
INFO:     Stopping reloader process [25519]


In [ ]:
!pip install fastapi uvicorn pyngrok --quiet
from fastapi import FastAPI
from pyngrok import ngrok
import uvicorn
import nest_asyncio

# Initialize app
app = FastAPI()

@app.get("/")
def home():
    return {"status": "API is running"}

# Configure ngrok (replace with your token)
NGROK_TOKEN = "*******************************************************"  # ← Paste your token here
ngrok.set_auth_token(NGROK_TOKEN)

# Start tunnel
public_url = ngrok.connect(8000).public_url
print(f"🌐 Public URL: {public_url}")

# Start FastAPI
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
!pip install fastapi uvicorn pyngrok nest-asyncio python-multipart --quiet

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional
from pyngrok import ngrok
import uvicorn
import nest_asyncio
import threading
from datetime import datetime

# ====== FASTAPI SETUP ======
app = FastAPI(title="Tax Chatbot API", version="1.0")

# ====== TAX ENDPOINTS ======
class TaxRequest(BaseModel):
    income: float
    age: int
    investments: dict = {"80C": 0, "NPS": 0}
    health_insurance: bool = False

@app.post("/calculate")
def calculate_tax(data: TaxRequest):
    """Real Indian tax calculation"""
    # Deductions
    deductions = {
        '80C': min(150000, data.investments.get("80C", 0)),
        '80D': 50000 if data.age > 60 else 25000 if data.health_insurance else 0,
        'NPS': min(50000, data.investments.get("NPS", 0))
    }

    # Tax slabs (2024-25)
    taxable_income = data.income - sum(deductions.values())
    tax = 0
    for min_val, max_val, rate in [
        (0, 300000, 0),
        (300001, 600000, 0.05),
        (600001, 900000, 0.10),
        (900001, 1200000, 0.15),
        (1200001, 1500000, 0.20),
        (1500001, float('inf'), 0.30)
    ]:
        if taxable_income > min_val:
            tax += (min(taxable_income, max_val) - min_val) * rate

    return {
        "taxable_income": taxable_income,
        "total_tax": tax,
        "effective_rate": (tax/data.income)*100,
        "deductions": deductions
    }

# ====== SERVER LAUNCH ======
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start in background
thread = threading.Thread(target=run_server, daemon=True)
thread.start()

ngrok.set_auth_token("*******************************************************")  # Get from https://dashboard.ngrok.com
public_url = ngrok.connect(8000).public_url

print(f"\n✅ Tax API Ready!\n"
      f"• Test URL: {public_url}/calculate\n"
      f"• Docs: {public_url}/docs\n\n"
      "Keep this cell running to maintain the connection.")

# Keep alive
while True: pass

INFO:     Started server process [3004]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



✅ Tax API Ready!
• Test URL: https://0264-34-125-68-33.ngrok-free.app/calculate
• Docs: https://0264-34-125-68-33.ngrok-free.app/docs

Keep this cell running to maintain the connection.


In [ ]:
# Prevent Colab timeouts
from IPython.display import Javascript
Javascript("""
function keepAlive() {
    console.log("Keeping session alive");
}
setInterval(keepAlive, 60000);
""")

In [ ]:
# ====== IMPORTS ======
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Dict

# ====== APP SETUP ======
app = FastAPI(title="Tax Calculator API", version="1.0")

# ====== DATA MODELS ======
class TaxRequest(BaseModel):
    income: float
    age: int
    investments: Dict[str, float] = {"80C": 0, "NPS": 0}
    health_insurance: bool = False

# ====== TAX CALCULATION LOGIC ======
def calculate_tax_from_slabs(income: float) -> float:
    """Calculate tax based on Indian income tax slabs (2024-25)"""
    tax = 0
    for min_val, max_val, rate in [
        (0, 300000, 0),
        (300001, 600000, 0.05),
        (600001, 900000, 0.10),
        (900001, 1200000, 0.15),
        (1200001, 1500000, 0.20),
        (1500001, float('inf'), 0.30)
    ]:
        if income > min_val:
            taxable = min(income, max_val) - min_val
            tax += taxable * rate
    return tax

# ====== API ENDPOINT ======
@app.post("/calculate")
def calculate_tax(data: TaxRequest):
    # Validate investments keys
    if not all(k in ["80C", "NPS"] for k in data.investments.keys()):
        raise HTTPException(
            status_code=422,
            detail="Investments must contain only '80C' and/or 'NPS' keys"
        )

    # Calculate deductions
    deductions = {
        '80C': min(150000, data.investments.get("80C", 0)),
        '80D': 50000 if data.age > 60 else 25000 if data.health_insurance else 0,
        'NPS': min(50000, data.investments.get("NPS", 0))
    }

    # Tax calculation
    taxable_income = data.income - sum(deductions.values())
    total_tax = calculate_tax_from_slabs(taxable_income)

    return {
        "taxable_income": taxable_income,
        "total_tax": total_tax,
        "effective_rate": (total_tax / data.income) * 100,
        "deductions": deductions
    }

# ====== ROOT ENDPOINT ======
@app.get("/")
def home():
    return {"message": "Tax Calculator API - Use /calculate endpoint"}
!curl -X POST "http://localhost:8000/calculate" \
-H "Content-Type: application/json" \
-d '{"income":1500000,"age":35,"investments":{"80C":150000,"NPS":50000},"health_insurance":true}'

curl: (7) Failed to connect to localhost port 8000 after 0 ms: Connection refused


In [ ]:
# ====== 1. INSTALL DEPENDENCIES ======
!pip install fastapi uvicorn pyngrok nest-asyncio python-multipart --quiet
!pkill -f "uvicorn"  # Kill any existing servers

# ====== 2. FASTAPI SETUP ======
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Dict
import uvicorn
from pyngrok import ngrok
import nest_asyncio
import threading
import time

app = FastAPI(
    title="Indian Tax Calculator API",
    version="2.0",
    description="Accurate Indian tax calculations with slab rates (2024-25)"
)

# ====== 3. TAX CALCULATION ======
class TaxRequest(BaseModel):
    income: float
    age: int
    investments: Dict[str, float] = {"80C": 0, "NPS": 0}
    health_insurance: bool = False

def calculate_tax_from_slabs(income: float) -> float:
    tax = 0
    for min_val, max_val, rate in [
        (0, 300000, 0), (300001, 600000, 0.05),
        (600001, 900000, 0.10), (900001, 1200000, 0.15),
        (1200001, 1500000, 0.20), (1500001, float('inf'), 0.30)
    ]:
        if income > min_val:
            tax += (min(income, max_val) - min_val) * rate
    return tax * 1.04  # 4% cess

@app.post("/calculate")
def calculate_tax(data: TaxRequest):
    # Validate input
    if data.income <= 0:
        raise HTTPException(400, "Income must be positive")

    # CORRECTED DEDUCTION CALCULATION
    deductions = {
        '80C': min(150000, data.investments.get("80C", 0)),  # Max ₹1.5L
        '80D': 50000 if data.age > 60 else 25000 if data.health_insurance else 0,  # Senior citizen check
        'NPS': min(50000, data.investments.get("NPS", 0))  # Max ₹50K
    }

    taxable_income = max(0, data.income - sum(deductions.values()))
    tax = calculate_tax_from_slabs(taxable_income)

    # ENSURE CORRECT FIELD NAMES IN RESPONSE
    return {
        "taxable_income": taxable_income,
        "total_tax": tax,
        "effective_rate": (tax / data.income) * 100,
        "deductions": {
            "80C": deductions['80C'],  # Will now correctly show "80C"
            "80D": deductions['80D'],  # Will now correctly show "80D"
            "NPS": deductions['NPS']
        }
    }
# ====== 4. SERVER LAUNCH ======
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="warning")

# Cleanup and start
ngrok.kill()  # Clear old tunnels
ngrok.set_auth_token("*******************************************************")
public_url = ngrok.connect(8000).public_url

nest_asyncio.apply()
thread = threading.Thread(target=run_server, daemon=True)
thread.start()

# Wait for server startup
time.sleep(3)

print(f"""
✅ API READY!
• Local: http://localhost:8000
• Public: {public_url}
• Docs: {public_url}/docs

Test with:
curl -X POST '{public_url}/calculate' \\
  -H 'Content-Type: application/json' \\
  -d '{{"income":1500000,"age":35,"investments":{{"80C":150000,"NPS":50000}},"health_insurance":true}}'
""")

# Keep alive without while True
from IPython.display import display, Javascript
display(Javascript('''
function keepAlive() {
    fetch("http://localhost:8000").catch(console.log)
}
setInterval(keepAlive, 30000)
'''))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00

✅ API READY!
• Local: http://localhost:8000
• Public: https://86ce-34-125-68-33.ngrok-free.app
• Docs: https://86ce-34-125-68-33.ngrok-free.app/docs

Test with:
curl -X POST 'https://86ce-34-125-68-33.ngrok-free.app/calculate' \
  -H 'Content-Type: application/json' \
  -d '{"income":1500000,"age":35,"investments":{"80C":150000,"NPS":50000},"health_insurance":true}'



<IPython.core.display.Javascript object>

In [ ]:
%%writefile frontend.html
<!DOCTYPE html>
<html>
<head>
    <title>Tax Calculator</title>
    <style>
        body { font-family: Arial; max-width: 500px; margin: 0 auto; padding: 20px; }
        input, button { padding: 8px; margin: 5px 0; width: 100%; }
        button { background: #4CAF50; color: white; border: none; cursor: pointer; }
        #result { margin-top: 20px; padding: 10px; background: #f5f5f5; }
    </style>
</head>
<body>
    <h2>Tax Calculator</h2>
    <form id="taxForm">
        <input type="number" name="income" placeholder="Annual Income (₹)" required>
        <input type="number" name="age" placeholder="Age" required>
        <h3>Investments</h3>
        <input type="number" name="80C" placeholder="80C Investment (Max ₹1.5L)">
        <input type="number" name="NPS" placeholder="NPS Investment (Max ₹50K)">
        <label>
            <input type="checkbox" name="health_insurance"> Health Insurance?
        </label>
        <button type="submit">Calculate Tax</button>
    </form>
    <div id="result"></div>

    <script>
        document.getElementById("taxForm").onsubmit = async (e) => {
            e.preventDefault();
            const formData = new FormData(e.target);

            const response = await fetch("https://YOUR_NGROK_URL/calculate", {
                method: "POST",
                headers: { "Content-Type": "application/json" },
                body: JSON.stringify({
                    income: parseFloat(formData.get("income")),
                    age: parseInt(formData.get("age")),
                    investments: {
                        "80C": parseFloat(formData.get("80C") || 0,
                        "NPS": parseFloat(formData.get("NPS") || 0
                    },
                    health_insurance: formData.get("health_insurance") === "on"
                })
            });

            const data = await response.json();
            document.getElementById("result").innerHTML = `
                <h3>Results</h3>
                <p>Taxable Income: ₹${data.taxable_income.toLocaleString()}</p>
                <p>Total Tax: ₹${data.total_tax.toLocaleString()}</p>
                <p>Effective Rate: ${data.effective_rate.toFixed(2)}%</p>
                <h4>Deductions Applied:</h4>
                <ul>
                    <li>80C: ₹${data.deductions['80C'].toLocaleString()}</li>
                    <li>80D: ₹${data.deductions['80D'].toLocaleString()}</li>
                    <li>NPS: ₹${data.deductions['NPS'].toLocaleString()}</li>
                </ul>
            `;
        };
    </script>
</body>
</html>

Overwriting frontend.html


In [ ]:
from google.colab.output import eval_js
from IPython.display import display, HTML

display(HTML('''
<script>
    window.open(" https://15a1-34-125-68-33.ngrok-free.app/docs", "_blank");
</script>
'''))

# Serve the frontend
display(HTML(filename='frontend.html'))

In [ ]:
# In Colab (after restarting runtime)
!npm install -g localtunnel
!lt --port 8000 --subdomain taxapi  # Will give you: https://taxapi.loca.lt

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 3s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴your url is: https://taxapi.loca.lt
